In [1]:
import pandas as pd
import anndata
import math
import os

# Define an output directory for the section files
output_dir = r"D:\Mouse\Matrices"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# Load cell metadata
metadata = pd.read_csv(
    r"D:\Mouse\GeneExpressions\metadata\MERFISH-C57BL6J-638850-CCF\20231215\views\cell_metadata_with_parcellation_annotation.csv",
    usecols=['cell_label','brain_section_label','neurotransmitter', 'class','subclass',
             'parcellation_index', 'parcellation_division', 'parcellation_structure', 'parcellation_substructure',
             'cluster_alias','average_correlation_score',
             'x_reconstructed','y_reconstructed','z_reconstructed']
             )



Empty DataFrame
Columns: [cell_label, brain_section_label, cluster_alias, average_correlation_score, neurotransmitter, class, subclass, x_reconstructed, y_reconstructed, z_reconstructed, parcellation_index, parcellation_division, parcellation_structure, parcellation_substructure]
Index: []


In [4]:
print(metadata[metadata['parcellation_division']=='Isocortex'].shape)

(935742, 14)


In [2]:
# Load cell metadata
metadata = pd.read_csv(
    r"D:\Mouse\GeneExpressions\metadata\MERFISH-C57BL6J-638850-CCF\20231215\views\cell_metadata_with_parcellation_annotation.csv",
    usecols=['cell_label','brain_section_label','neurotransmitter', 'class','subclass',
             'parcellation_index', 'parcellation_division', 'parcellation_structure', 'parcellation_substructure',
             'cluster_alias','average_correlation_score',
             'x_reconstructed','y_reconstructed','z_reconstructed']
)

# Set the index to 'cell_label' so it matches exprData.obs_names
metadata_indexed = metadata.set_index('cell_label')
columns_to_join = ['neurotransmitter', 'class','subclass',
                    'parcellation_index', 'parcellation_division', 'parcellation_structure', 'parcellation_substructure',
                    'cluster_alias','average_correlation_score',
                    'x_reconstructed','y_reconstructed','z_reconstructed']
metadata_indexed = metadata_indexed[columns_to_join]

# Get the list of brain sections
brain_sections = sorted(metadata['brain_section_label'].unique())

# Load the expression data
geneImputedExpression = r"D:\Mouse\GeneExpressions\expression_matrices\MERFISH-C57BL6J-638850-imputed\20240831\C57BL6J-638850-imputed-log2.h5ad"
exprData = anndata.read_h5ad(geneImputedExpression, backed='r')

In [3]:
# Process each brain section
for brain_section in brain_sections:
    print(f"Processing {brain_section}")
    
    # Get cell labels for the current brain section from the metadata
    cellsMet = metadata.loc[metadata['brain_section_label'] == brain_section, 'cell_label'].tolist()
    print(f"Found {len(cellsMet)} cells in {brain_section}")
    
    # Find the intersection between these cell labels and the AnnData obs_names
    cellsExpr = exprData.obs_names.intersection(cellsMet)
    print(f"Found {len(cellsExpr)} cells in the expression data")
    
    # Determine chunking parameters (process 1/20 of cells at a time)
    n_cells = len(cellsExpr)
    n_chunks = 12
    chunk_size = math.ceil(n_cells / n_chunks)
    chunk_files = []
    
    # Process cells in chunks
    for i in range(n_chunks):
        start = i * chunk_size
        end = min((i + 1) * chunk_size, n_cells)
        chunk_cell_names = cellsExpr[start:end].tolist()
        if not chunk_cell_names:
            break
        print(f"Processing chunk {i+1} with {len(chunk_cell_names)} cells.")
        
        # Subset the expression data for these cells and load into memory
        chunkData = exprData[chunk_cell_names, :].to_memory()
        
        # Combine metadata with the AnnData object's .obs dataframe
        chunkData.obs = chunkData.obs.join(metadata_indexed, how='left')
        
        # Convert problematic columns (e.g., 'neurotransmitter') to string to avoid h5py errors
        for col in chunkData.obs.select_dtypes(include=['object']).columns:
            chunkData.obs[col] = chunkData.obs[col].astype(str)
        
        # Define a temporary filename for this chunk and save it
        chunk_filename = os.path.join(output_dir, f"{brain_section}_chunk_{i+1}.h5ad")
        chunkData.write(chunk_filename)
        chunk_files.append(chunk_filename)
        print(f"Saved {chunk_filename}")
    
    # Now, load all chunk files and concatenate them into one AnnData object
    chunk_adatas = [anndata.read_h5ad(chunk_file) for chunk_file in chunk_files]
    combined_sectionData = anndata.concat(chunk_adatas, join='outer', merge='same')
    
    # Save the combined section data
    final_filename = os.path.join(output_dir, f"{brain_section}.h5ad")
    combined_sectionData.write(final_filename)
    print(f"Saved combined section data as {final_filename}")
    
    # Delete temporary chunk files
    for chunk_file in chunk_files:
        os.remove(chunk_file)
        print(f"Deleted temporary file {chunk_file}")


Processing C57BL6J-638850.05
Found 45915 cells in C57BL6J-638850.05
Found 45915 cells in the expression data
Processing chunk 1 with 3827 cells.
Saved D:\Mouse\Matrices\C57BL6J-638850.05_chunk_1.h5ad
Processing chunk 2 with 3827 cells.
Saved D:\Mouse\Matrices\C57BL6J-638850.05_chunk_2.h5ad
Processing chunk 3 with 3827 cells.
Saved D:\Mouse\Matrices\C57BL6J-638850.05_chunk_3.h5ad
Processing chunk 4 with 3827 cells.
Saved D:\Mouse\Matrices\C57BL6J-638850.05_chunk_4.h5ad
Processing chunk 5 with 3827 cells.
Saved D:\Mouse\Matrices\C57BL6J-638850.05_chunk_5.h5ad
Processing chunk 6 with 3827 cells.
Saved D:\Mouse\Matrices\C57BL6J-638850.05_chunk_6.h5ad
Processing chunk 7 with 3827 cells.
Saved D:\Mouse\Matrices\C57BL6J-638850.05_chunk_7.h5ad
Processing chunk 8 with 3827 cells.
Saved D:\Mouse\Matrices\C57BL6J-638850.05_chunk_8.h5ad
Processing chunk 9 with 3827 cells.
Saved D:\Mouse\Matrices\C57BL6J-638850.05_chunk_9.h5ad
Processing chunk 10 with 3827 cells.
Saved D:\Mouse\Matrices\C57BL6J-638

In [6]:
# Load the .h5ad file
adata = anndata.read_h5ad(r"D:\Mouse\Matrices\C57BL6J-638850.05.h5ad")

# Print the shape of the data matrix (observations x variables)
print("Shape (observations, variables):", adata.shape)

# Print a summary of the AnnData object
print(adata)

Shape (observations, variables): (45915, 8460)
AnnData object with n_obs × n_vars = 45915 × 8460
    obs: 'brain_section_label', 'neurotransmitter', 'class', 'subclass', 'parcellation_index', 'parcellation_division', 'parcellation_structure', 'parcellation_substructure', 'cluster_alias', 'average_correlation_score', 'x_reconstructed', 'y_reconstructed', 'z_reconstructed'
    var: 'gene_symbol'


In [7]:
# Explore the structure:
print("Observation metadata (obs):")
print(adata.obs.head())

Observation metadata (obs):
                    brain_section_label                     subclass  \
cell_label                                                             
1019171911100960969   C57BL6J-638850.05  106 PVpo-VMPO-MPN Hmx2 Gaba   
1019171911101270462   C57BL6J-638850.05     216 MB-MY Tph2 Glut-Sero   
1019171911101270488   C57BL6J-638850.05     216 MB-MY Tph2 Glut-Sero   
1019171911101270469   C57BL6J-638850.05     216 MB-MY Tph2 Glut-Sero   
1019171911101270478   C57BL6J-638850.05     216 MB-MY Tph2 Glut-Sero   

                    parcellation_structure  x_reconstructed  y_reconstructed  \
cell_label                                                                     
1019171911100960969                    NTS         5.013879         6.247225   
1019171911101270462          MY-unassigned         5.442881         7.231411   
1019171911101270488          MY-unassigned         5.424471         7.249722   
1019171911101270469          MY-unassigned         5.424150        

In [8]:
print("\nVariable metadata (var):")
print(adata.var.head())


Variable metadata (var):
                   gene_symbol
gene_identifier               
ENSMUSG00000026676       Ccdc3
ENSMUSG00000024517         Grp
ENSMUSG00000029361        Nos1
ENSMUSG00000062372        Otof
ENSMUSG00000055639       Dach1


In [9]:
print("\nUnstructured annotations (uns) keys:")
print(adata.uns.keys())


Unstructured annotations (uns) keys:
odict_keys([])
